In [1]:
from source_files import *

import query_helpers

import math
import numpy as np
import pandas as pd
import numba

In [2]:
@numba.jit
def column_to_volume(snow_depth, x_resolution, y_resolution):
    length = len(snow_depth)
    result = np.empty(length, dtype=snow_depth.dtype)
    for i in range(length):
        result[i] = snow_volume(snow_depth[i], x_resolution, y_resolution)
    return result

@numba.jit
def snow_volume(snow_depth, x_resolution, y_resolution):
    if snow_depth < 0:
        return 0.0
    else:
        return math.fabs(x_resolution * y_resolution) * snow_depth

In [3]:
df = db_query_to_df(
        'Select aso_snow_depth, sfm_snow_depth, sd_difference, casi_class FROM ERW_analysis'
)

df['aso_snow_volume'] = column_to_volume(
    df.aso_snow_depth.to_numpy(), 
    aso_snow_depth.x_resolution, 
    aso_snow_depth.y_resolution
)
df['sfm_snow_volume'] = column_to_volume(
    df.sfm_snow_depth.to_numpy(), 
    sfm_snow_depth.x_resolution, 
    sfm_snow_depth.y_resolution
)

In [4]:
positive_sfm = query_helpers.get_positive(df, 'sfm_snow_depth')
negative_sfm = query_helpers.get_negative(df, 'sfm_snow_depth')
no_values_sfm = query_helpers.get_no_data(df, 'sfm_snow_depth')

columns = ['casi_class', 'sfm_snow_depth', 'aso_snow_depth', 'sd_difference']

pd.set_option('display.float_format', lambda x: '%.2f m' % x)

# Snow Depth

In [5]:
df[columns].agg([np.mean, np.median, np.std])

,sfm_snow_depth,aso_snow_depth,sd_difference
mean,-0.55 m,0.89 m,-1.44 m
median,0.45 m,0.64 m,-0.17 m
std,4.31 m,0.88 m,4.15 m


In [6]:
df[columns].groupby('casi_class').agg([np.mean, np.median, np.std])

sfm_snow_depth                aso_snow_depth                \
                     mean  median    std           mean median    std   
casi_class                                                              
Snow               0.72 m  0.68 m 1.81 m         1.08 m 0.83 m 0.96 m   
Vegetation        -5.70 m -3.21 m 7.37 m         0.50 m 0.45 m 0.33 m   
Rock              -0.97 m  0.10 m 3.99 m         0.45 m 0.27 m 0.54 m   
Water             -4.67 m -1.10 m 7.21 m         0.47 m 0.40 m 0.36 m   

           sd_difference                 
                    mean  median    std  
casi_class                               
Snow             -0.36 m -0.12 m 1.54 m  
Vegetation       -6.20 m -3.68 m 7.45 m  
Rock             -1.42 m -0.15 m 4.02 m  
Water            -5.13 m -1.56 m 7.29 m

### Positive SfM values

In [7]:
positive_sfm[columns].agg([np.mean, np.median, np.std])

,sfm_snow_depth,aso_snow_depth,sd_difference
mean,1.06 m,1.05 m,0.01 m
median,0.76 m,0.79 m,-0.03 m
std,1.11 m,0.96 m,0.83 m


In [8]:
positive_sfm[columns].groupby('casi_class').agg([np.mean, np.median, np.std])

sfm_snow_depth               aso_snow_depth                \
                     mean median    std           mean median    std   
casi_class                                                             
Snow               1.08 m 0.83 m 0.97 m         1.18 m 0.93 m 0.99 m   
Vegetation         1.54 m 0.73 m 2.21 m         0.45 m 0.37 m 0.36 m   
Rock               0.62 m 0.38 m 0.84 m         0.52 m 0.31 m 0.63 m   
Water              1.45 m 0.63 m 2.21 m         0.43 m 0.33 m 0.43 m   

           sd_difference                 
                    mean  median    std  
casi_class                               
Snow             -0.09 m -0.06 m 0.48 m  
Vegetation        1.09 m  0.26 m 2.24 m  
Rock              0.10 m  0.04 m 0.74 m  
Water             1.02 m  0.23 m 2.23 m

In [37]:
values = (df.aso_snow_depth < 0.08).value_counts()
values[True]/values.sum()

0.08178103810758588

In [38]:
values = (positive_sfm.sfm_snow_depth <= 0.22).value_counts()
values[True]/values.sum()

0.15113021384267192

In [77]:
df.where(df.aso_snow_depth <= 0.08)[['sfm_snow_depth', 'aso_snow_depth', 'sd_difference']].agg([np.mean, np.median, np.std])

,sfm_snow_depth,aso_snow_depth,sd_difference
mean,-0.143425,0.039942,-0.183352
median,0.035156,0.038855,-0.003943
std,1.718449,0.023104,1.719109


In [60]:
for i in range(1, 6):
    print(f"Depth range: {i - 1} m and {i} m")
    query = positive_sfm.where(((i - 1) < positive_sfm.sfm_snow_depth) & (positive_sfm.sfm_snow_depth <= i))
    print(query[['sfm_snow_depth', 'aso_snow_depth', 'sd_difference']].agg([np.mean, np.median, np.std]))
    #     print(f" SfM: {query.sfm_snow_volume.sum():,.2f}")
#     print(f" ASO: {query.aso_snow_volume.sum():,.2f}")

Depth range: 0 m and 1 m
        sfm_snow_depth  aso_snow_depth  sd_difference
mean          0.464389        0.579376      -0.114988
median        0.447510        0.517126      -0.053503
std           0.279252        0.432241       0.387503
Depth range: 1 m and 2 m
        sfm_snow_depth  aso_snow_depth  sd_difference
mean          1.404407        1.383389       0.021017
median        1.361084        1.366980       0.009729
std           0.279142        0.556227       0.494507
Depth range: 2 m and 3 m
        sfm_snow_depth  aso_snow_depth  sd_difference
mean          2.411041        2.258940       0.152100
median        2.368896        2.349890       0.025598
std           0.281298        0.801056       0.771067
Depth range: 3 m and 4 m
        sfm_snow_depth  aso_snow_depth  sd_difference
mean          3.415680        2.976354       0.439325
median        3.375732        3.293982       0.074915
std           0.281670        1.220017       1.213817
Depth range: 4 m and 5 m
        sfm

### Negative SfM values

In [10]:
negative_sfm[columns].agg([np.mean, np.median, np.std])

,sfm_snow_depth,aso_snow_depth,sd_difference
mean,-4.81 m,0.48 m,-5.28 m
median,-0.93 m,0.39 m,-1.67 m
std,6.30 m,0.42 m,6.39 m


In [81]:
negative_sfm.where(negative_sfm.aso_snow_depth <= 0.08)[['sfm_snow_depth', 'aso_snow_depth', 'sd_difference']].agg([np.mean, np.median, np.std])

,sfm_snow_depth,aso_snow_depth,sd_difference
mean,-0.779916,0.039877,-0.819759
median,-0.180664,0.038611,-0.220984
std,2.223287,0.023195,2.224483


In [11]:
negative_sfm[columns].groupby('casi_class').agg([np.mean, np.median, np.std])

sfm_snow_depth                aso_snow_depth                \
                     mean  median    std           mean median    std   
casi_class                                                              
Snow              -1.41 m -0.28 m 3.42 m         0.52 m 0.36 m 0.52 m   
Vegetation        -8.76 m -8.57 m 6.61 m         0.52 m 0.48 m 0.31 m   
Rock              -3.30 m -0.43 m 5.40 m         0.35 m 0.23 m 0.38 m   
Water             -8.12 m -7.17 m 6.76 m         0.49 m 0.44 m 0.32 m   

           sd_difference                 
                    mean  median    std  
casi_class                               
Snow             -1.91 m -0.78 m 3.48 m  
Vegetation       -9.28 m -9.11 m 6.71 m  
Rock             -3.65 m -0.80 m 5.53 m  
Water            -8.60 m -7.66 m 6.86 m

In [12]:
pd.set_option('display.float_format', None)

# Snow Volume

In [13]:
m2_style = "{:,.0f} m<sup>3</sup>"
table_style = {
    'sfm_snow_volume': m2_style,
    'aso_snow_volume': m2_style,
    'difference': m2_style,
    'sfm % to aso': "{:.2%}",
    'percent_sfm_scene': "{:.2%}",
}
columns = ['casi_class', 'aso_snow_volume', 'sfm_snow_volume']

In [14]:
total_volume = pd.DataFrame({
    'sfm_snow_volume': df.sfm_snow_volume.sum(),
    'aso_snow_volume': df.aso_snow_volume.sum(),
    },
    index=[0]
)
total_volume['difference'] = total_volume.aso_snow_volume - total_volume.sfm_snow_volume
total_volume['sfm % to aso'] = total_volume.sfm_snow_volume / total_volume.aso_snow_volume
total_volume.style.format(table_style)

,sfm_snow_volume,aso_snow_volume,difference,sfm % to aso
0,"21,103,260 m3","24,523,900 m3","3,420,640 m3",86.05%


In [15]:
captured_volume = pd.DataFrame({
    'sfm_snow_volume': positive_sfm.sfm_snow_volume.sum(),
    'aso_snow_volume': positive_sfm.aso_snow_volume.sum(),
    },
    index=[0]
)
captured_volume['difference'] = captured_volume.aso_snow_volume - captured_volume.sfm_snow_volume
captured_volume['sfm % to aso'] = captured_volume.sfm_snow_volume / captured_volume.aso_snow_volume
captured_volume.style.format(table_style)

,sfm_snow_volume,aso_snow_volume,difference,sfm % to aso
0,"21,103,260 m3","20,840,970 m3","-262,289 m3",101.26%


In [16]:
missed_volume = pd.DataFrame({
    'sfm_snow_volume': negative_sfm.sfm_snow_volume.sum(),
    'aso_snow_volume': negative_sfm.aso_snow_volume.sum(),
    },
    index=[0]
)
missed_volume['difference'] = missed_volume.aso_snow_volume - missed_volume.sfm_snow_volume
missed_volume['sfm % to aso'] = missed_volume.sfm_snow_volume / missed_volume.aso_snow_volume
missed_volume.style.format(table_style)

,sfm_snow_volume,aso_snow_volume,difference,sfm % to aso
0,0 m3,"3,682,930 m3","3,682,930 m3",0.00%


In [17]:
assert (captured_volume.aso_snow_volume + missed_volume.aso_snow_volume).item(), total_volume.aso_snow_volume.item()

In [18]:
num_pixels_aso = df.aso_snow_depth.count()
num_pixels_sfm = positive_sfm.aso_snow_depth.count()

In [36]:
print("Pixels with depth:")
print(f"  ASO count: {num_pixels_aso:,}")
print(f"  SfM count: {num_pixels_sfm:,}")
print("Percent ASO pixels:")
print(f"  with values in SfM: {num_pixels_sfm/num_pixels_aso:.2%}")
print(f"  with negative in SfM: {negative_sfm.query('sfm_snow_depth == sfm_snow_depth').aso_snow_depth.count()/num_pixels_aso:.2%}")
print(f"  with no value in SfM: {no_values_sfm.aso_snow_depth.count()/num_pixels_aso:.2%}")

Pixel:
  ASO count: 27607023
  SfM count: 19942465
Percent ASO pixels:
  with values in SfM: 72.24%
  with negative in SfM: 27.25%
  with no value in SfM: 0.51%


In [20]:
pixel_stats = positive_sfm[columns].groupby('casi_class').count()

grouped_volume = df[columns].groupby('casi_class').sum()
grouped_volume['difference'] = grouped_volume.aso_snow_volume - grouped_volume.sfm_snow_volume
grouped_volume['sfm % to aso'] = grouped_volume.sfm_snow_volume / grouped_volume.aso_snow_volume
grouped_volume['percent_sfm_scene'] = (pixel_stats.sfm_snow_volume / num_pixels_sfm)
grouped_volume.style.format(table_style)

,aso_snow_volume,sfm_snow_volume,difference,sfm % to aso,percent_sfm_scene
casi_class,,,,,
Snow,"20,450,122 m3","17,465,415 m3","2,984,707 m3",85.40%,80.83%
Vegetation,"1,759,627 m3","1,615,518 m3","144,108 m3",91.81%,5.25%
Rock,"1,852,569 m3","1,510,543 m3","342,026 m3",81.54%,12.15%
Water,"461,582 m3","511,783 m3","-50,201 m3",110.88%,1.77%
